# Clasificación multiclase

## Introduction

En este ejercicio se implementa la regresion one-vs-all y una red neuronal para reconocimiento de digitos.



# LABORATORIO 02 Regresion logistica one vs all, multiclase
Diego Roberto Arancibia Delgado
https://github.com/drArancibiaDelgado/sis420-/tree/main/Laboratorios/Lab%203

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

import pandas as pd

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize


from sklearn.model_selection import train_test_split
import tabulate
# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

## Clasificación multiclase

In [4]:
#nuestro dataset contiene 3 clases la cuales son (galaxias, estrellas y quasares)
# La entrada es de 14 elemento contando con x0
input_layer_size  = 40

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 3

#  datos de entrenamiento almacenados en los arreglos X, y
# data = loadmat(os.path.join('Datasets', 'ex3data1.mat'))
data = pd.read_csv("SDSS_DR18.csv")


data

,objid,specobjid,ra,dec,u,g,r,i,z,run,...,psfMag_g,psfMag_i,psfMag_z,expAB_u,expAB_g,expAB_r,expAB_i,expAB_z,redshift,class
0,1.240000e+18,3.240000e+17,184.950869,0.733068,18.87062,17.59612,17.11245,16.83899,16.70908,756,...,19.96352,19.25145,19.05230,0.479021,0.518483,0.520474,0.508502,0.488969,0.041691,GALAXY
1,1.240000e+18,3.250000e+17,185.729201,0.679704,19.59560,19.92153,20.34448,20.66213,20.59599,756,...,19.92417,20.65535,20.57387,0.573926,0.531728,0.403072,0.999874,0.189495,-0.000814,STAR
2,1.240000e+18,3.240000e+17,185.687690,0.823480,19.26421,17.87891,17.09593,16.65159,16.35329,756,...,19.33645,18.16669,17.78844,0.701666,0.743386,0.770897,0.778642,0.736771,0.113069,GALAXY
3,1.240000e+18,2.880000e+18,185.677904,0.768362,19.49739,17.96166,17.41269,17.20545,17.11567,756,...,17.96176,17.21564,17.12367,0.999818,0.787760,0.745611,0.399718,0.986137,0.000087,STAR
4,1.240000e+18,2.880000e+18,185.814763,0.776940,18.31519,16.83033,16.26352,16.06320,15.97527,756,...,16.85104,16.08275,15.98694,0.999795,0.834450,0.723526,0.712259,0.527055,0.000018,STAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.240000e+18,3.580000e+18,154.077143,55.614066,19.39861,18.35476,18.00348,17.89408,17.81222,2243,...,18.35865,17.90406,17.82489,0.999653,0.999791,0.794516,0.802310,0.537987,-0.000101,STAR
99996,1.240000e+18,3.580000e+18,154.067926,55.635794,19.07703,18.05159,17.78332,17.68976,17.66209,2243,...,18.06198,17.69745,17.67005,0.802407,0.999912,0.999776,0.999682,0.544329,-0.000352,STAR
99997,1.240000e+18,1.070000e+18,153.897018,55.712582,19.07982,17.51349,16.64037,16.24183,15.91180,2243,...,19.05242,17.79391,17.40130,0.859021,0.760212,0.769053,0.772322,0.845797,0.117501,GALAXY
99998,1.240000e+18,6.950000e+17,235.656141,56.297044,17.27528,16.41704,16.11662,15.98858,15.97745,2206,...,16.42854,15.98151,15.97900,0.655041,0.199795,0.481020,0.650008,0.822835,-0.000400,STAR


In [5]:
data = data.drop(["rerun", "objid"], axis=1)
data

,specobjid,ra,dec,u,g,r,i,z,run,camcol,...,psfMag_g,psfMag_i,psfMag_z,expAB_u,expAB_g,expAB_r,expAB_i,expAB_z,redshift,class
0,3.240000e+17,184.950869,0.733068,18.87062,17.59612,17.11245,16.83899,16.70908,756,5,...,19.96352,19.25145,19.05230,0.479021,0.518483,0.520474,0.508502,0.488969,0.041691,GALAXY
1,3.250000e+17,185.729201,0.679704,19.59560,19.92153,20.34448,20.66213,20.59599,756,5,...,19.92417,20.65535,20.57387,0.573926,0.531728,0.403072,0.999874,0.189495,-0.000814,STAR
2,3.240000e+17,185.687690,0.823480,19.26421,17.87891,17.09593,16.65159,16.35329,756,5,...,19.33645,18.16669,17.78844,0.701666,0.743386,0.770897,0.778642,0.736771,0.113069,GALAXY
3,2.880000e+18,185.677904,0.768362,19.49739,17.96166,17.41269,17.20545,17.11567,756,5,...,17.96176,17.21564,17.12367,0.999818,0.787760,0.745611,0.399718,0.986137,0.000087,STAR
4,2.880000e+18,185.814763,0.776940,18.31519,16.83033,16.26352,16.06320,15.97527,756,5,...,16.85104,16.08275,15.98694,0.999795,0.834450,0.723526,0.712259,0.527055,0.000018,STAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3.580000e+18,154.077143,55.614066,19.39861,18.35476,18.00348,17.89408,17.81222,2243,3,...,18.35865,17.90406,17.82489,0.999653,0.999791,0.794516,0.802310,0.537987,-0.000101,STAR
99996,3.580000e+18,154.067926,55.635794,19.07703,18.05159,17.78332,17.68976,17.66209,2243,3,...,18.06198,17.69745,17.67005,0.802407,0.999912,0.999776,0.999682,0.544329,-0.000352,STAR
99997,1.070000e+18,153.897018,55.712582,19.07982,17.51349,16.64037,16.24183,15.91180,2243,3,...,19.05242,17.79391,17.40130,0.859021,0.760212,0.769053,0.772322,0.845797,0.117501,GALAXY
99998,6.950000e+17,235.656141,56.297044,17.27528,16.41704,16.11662,15.98858,15.97745,2206,3,...,16.42854,15.98151,15.97900,0.655041,0.199795,0.481020,0.650008,0.822835,-0.000400,STAR


In [6]:
class_mapping = {"GALAXY": 0, "QSO": 1, "STAR": 2}
data["class"] = data["class"].fillna(0)
data["class"] = data["class"].map(class_mapping)

data

,specobjid,ra,dec,u,g,r,i,z,run,camcol,...,psfMag_g,psfMag_i,psfMag_z,expAB_u,expAB_g,expAB_r,expAB_i,expAB_z,redshift,class
0,3.240000e+17,184.950869,0.733068,18.87062,17.59612,17.11245,16.83899,16.70908,756,5,...,19.96352,19.25145,19.05230,0.479021,0.518483,0.520474,0.508502,0.488969,0.041691,0
1,3.250000e+17,185.729201,0.679704,19.59560,19.92153,20.34448,20.66213,20.59599,756,5,...,19.92417,20.65535,20.57387,0.573926,0.531728,0.403072,0.999874,0.189495,-0.000814,2
2,3.240000e+17,185.687690,0.823480,19.26421,17.87891,17.09593,16.65159,16.35329,756,5,...,19.33645,18.16669,17.78844,0.701666,0.743386,0.770897,0.778642,0.736771,0.113069,0
3,2.880000e+18,185.677904,0.768362,19.49739,17.96166,17.41269,17.20545,17.11567,756,5,...,17.96176,17.21564,17.12367,0.999818,0.787760,0.745611,0.399718,0.986137,0.000087,2
4,2.880000e+18,185.814763,0.776940,18.31519,16.83033,16.26352,16.06320,15.97527,756,5,...,16.85104,16.08275,15.98694,0.999795,0.834450,0.723526,0.712259,0.527055,0.000018,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3.580000e+18,154.077143,55.614066,19.39861,18.35476,18.00348,17.89408,17.81222,2243,3,...,18.35865,17.90406,17.82489,0.999653,0.999791,0.794516,0.802310,0.537987,-0.000101,2
99996,3.580000e+18,154.067926,55.635794,19.07703,18.05159,17.78332,17.68976,17.66209,2243,3,...,18.06198,17.69745,17.67005,0.802407,0.999912,0.999776,0.999682,0.544329,-0.000352,2
99997,1.070000e+18,153.897018,55.712582,19.07982,17.51349,16.64037,16.24183,15.91180,2243,3,...,19.05242,17.79391,17.40130,0.859021,0.760212,0.769053,0.772322,0.845797,0.117501,0
99998,6.950000e+17,235.656141,56.297044,17.27528,16.41704,16.11662,15.98858,15.97745,2206,3,...,16.42854,15.98151,15.97900,0.655041,0.199795,0.481020,0.650008,0.822835,-0.000400,2


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 41 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   specobjid    100000 non-null  float64
 1   ra           100000 non-null  float64
 2   dec          100000 non-null  float64
 3   u            100000 non-null  float64
 4   g            100000 non-null  float64
 5   r            100000 non-null  float64
 6   i            100000 non-null  float64
 7   z            100000 non-null  float64
 8   run          100000 non-null  int64  
 9   camcol       100000 non-null  int64  
 10  field        100000 non-null  int64  
 11  plate        100000 non-null  int64  
 12  mjd          100000 non-null  int64  
 13  fiberid      100000 non-null  int64  
 14  petroRad_u   100000 non-null  float64
 15  petroRad_g   100000 non-null  float64
 16  petroRad_i   100000 non-null  float64
 17  petroRad_r   100000 non-null  float64
 18  petroRad_z   100000 non-n

In [8]:
value_counts = data["class"].value_counts()

print(value_counts)

class
0    52343
2    37232
1    10425
Name: count, dtype: int64


In [9]:
y_tem = data["class"]

data_class_0 = data[y_tem == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

data_class_1 = data[y_tem == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

data_class_2 = data[y_tem == 2]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)



In [10]:
print("para la clase 0 se tiene la cantidad de: ",data_class_0.shape[0], "para el 80 %: ", train_class_0.shape[0], "para el 20 %: ", test_class_0.shape[0])
print("para la clase 1 se tiene la cantidad de: ",data_class_1.shape[0], "para el 80 %: ", train_class_1.shape[0], "para el 20 %: ", test_class_1.shape[0])
print("para la clase 2 se tiene la cantidad de: ",data_class_2.shape[0], "para el 80 %: ", train_class_2.shape[0], "para el 20 %: ", test_class_2.shape[0])
print("la cantidad total de datos es: ", data.shape[0])

para la clase 0 se tiene la cantidad de:  52343 para el 80 %:  41874 para el 20 %:  10469
para la clase 1 se tiene la cantidad de:  10425 para el 80 %:  8340 para el 20 %:  2085
para la clase 2 se tiene la cantidad de:  37232 para el 80 %:  29785 para el 20 %:  7447
la cantidad total de datos es:  100000


In [11]:
X_train_class_0 = train_class_0.drop("class", axis=1)
y_train_class_0 = train_class_0["class"]

X_train_class_1 = train_class_1.drop("class", axis=1)
y_train_class_1 = train_class_1["class"]

X_train_class_2 = train_class_2.drop("class", axis=1)
y_train_class_2 = train_class_2["class"]

X_test_class_0 = test_class_0.drop("class", axis=1)
y_test_class_0 = test_class_0["class"]

X_test_class_1 = test_class_1.drop("class", axis=1)
y_test_class_1 = test_class_1["class"]

X_test_class_2 = test_class_2.drop("class", axis=1)
y_test_class_2 = test_class_2["class"]

In [12]:
X_train = pd.concat([X_train_class_0, X_train_class_1, X_train_class_2]).values
y_train = pd.concat([y_train_class_0, y_train_class_1, y_train_class_2]).values

indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = X_train.shape[0]


X_test = pd.concat([X_test_class_0, X_test_class_1, X_test_class_2]).values
y_test = pd.concat([y_test_class_0, y_test_class_1, y_test_class_2]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
m_test = X_test.shape[0]

# print(X_train)
# print(y_train)

In [13]:
#encontrar el numero de ejemplos de entrenamiento y el numero de caracteristicas
print("la cantidad de ejemplos para entrenamiento es: ", len(X_train))
print("la cantidad de ejemplos para pruebas es: ", len(X_test))
print("la cantidad de caracteristicas es: ", X_train.shape[1])
print("la cantidad de etiquetas es: ", num_labels)
print("la cantidad total de ejemplos: ", data.shape[0])

la cantidad de ejemplos para entrenamiento es:  79999
la cantidad de ejemplos para pruebas es:  20001
la cantidad de caracteristicas es:  40
la cantidad de etiquetas es:  3
la cantidad total de ejemplos:  100000


In [14]:
# X = data.iloc[:, 2:]
# y = data.iloc[:, 0]
# m = y.size

In [15]:
##aqui tratarmos las clases GALAXY, STAR, QSO
# data.iloc[:, 42] = data.iloc[:, 42].map({'GALAXY': 0, 'STAR': 1, 'QSO' :2 }).fillna(-1).astype(int)




In [16]:
# print(X.iloc[0,:])
# print(y)

In [17]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [18]:
X_norm, mu, sigma = featureNormalize(X_train)

In [38]:
#esto es simplemente para mostrar los datos
print('{:>8s}{:>8s}{:>10s}{:>15s}{:>8s}{:>10s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}{:>8s}'.format('X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]', 'X[:, 12]', 'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]', 'X[:, 20]', 'X[:, 21]', 'X[:, 22]', 'X[:, 23]', 'X[:, 24]', 'X[:, 25]', 'X[:, 26]', 'X[:, 27]', 'X[:, 28]', 'X[:, 29]', 'X[:, 30]', 'X[:, 31]', 'X[:, 32]', 'X[:, 33]', 'X[:, 34]', 'X[:, 35]', 'X[:, 36]', 'X[:, 37]', 'X[:, 38]', 'X[:, 39]', 'X[:, 40]', 'X[:, 41]', 'X[:, 42]', 'X[:, 43]'))
print('-'*200)
for i in range(10):
    print('{:8.3f}{:8.3f}{:10.3f}{:15.3f}{:15.3f}{:15.3f}{:15.3f}{:8.3f}{:8.3f}{:8.3f}{:10.3f}{:15.3f}{:15.3f}{:15.3f}{:15.3f}{:8.3f}{:8.3f}'.format(
        X_norm[i, 0], 
        X_norm[i, 1], 
        X_norm[i, 2], 
        X_norm[i, 3], 
        X_norm[i, 4], 
        X_norm[i, 5], 
        X_norm[i, 6], 
        X_norm[i, 7], 
        X_norm[i, 8], 
        X_norm[i, 9], 
        X_norm[i, 10], 
        X_norm[i, 11], 
        X_norm[i, 12], 
        X_norm[i, 13], 
        X_norm[i, 14], 
        X_norm[i, 15], 
        X_norm[i, 16],
        X_norm[i, 17],
        X_norm[i, 18],
        X_norm[i, 19],
        X_norm[i, 20],
        X_norm[i, 21],
        X_norm[i, 22],
        X_norm[i, 23],
        X_norm[i, 24],
        X_norm[i, 25],
        X_norm[i, 26],
        X_norm[i, 27],))

  X[:,0] X[:, 1]   X[:, 2]        X[:, 3] X[:, 4]   X[:, 5] X[:, 6] X[:, 7] X[:, 8] X[:, 9]X[:, 10]X[:, 11]X[:, 12]X[:, 13]X[:, 14]X[:, 15]X[:, 16]X[:, 17]X[:, 18]X[:, 19]X[:, 20]X[:, 21]X[:, 22]X[:, 23]X[:, 24]X[:, 25]X[:, 26]X[:, 27]X[:, 28]X[:, 29]X[:, 30]X[:, 31]X[:, 32]X[:, 33]X[:, 34]X[:, 35]X[:, 36]X[:, 37]X[:, 38]X[:, 39]X[:, 40]X[:, 41]X[:, 42]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  -0.688  -0.246    -0.721          0.520          0.025         -0.286         -0.447  -0.009  -0.521  -1.558    -0.314         -0.688         -0.925         -1.003         -0.027   0.342   0.280
  -0.559   0.388    -0.606          0.104         -0.367         -0.495         -0.557  -0.011  -0.045  -0.929     1.555         -0.559         -0.475         -1.489         -0.044   0.202   0.201
  -0.672   0.400    -0.653         -1.574        

In [20]:
# # llama featureNormalize con los datos cargados
# X = X.apply(pd.to_numeric, errors='coerce')
# X = X.dropna(axis=1)




In [21]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X_train.shape
# Agraga el termino de intercepción a A
# X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm.copy()
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

### Visualización de los datos

no se uso en esta caso por eso todo esta comentado 


In [22]:
# def displayData(X, example_width=None, figsize=(10, 10)):
#     """
#     Muestra datos 2D almacenados en X en una cuadrícula apropiada.
#     """
#     # Calcula filas, columnas
#     if X.ndim == 2:
#         m, n = X.shape
#     elif X.ndim == 1:
#         n = X.size
#         m = 1
#         X = X[None]  # Promocionar a una matriz bidimensional
#     else:
#         raise IndexError('La entrada X debe ser 1 o 2 dimensinal.')

#     example_width = example_width or int(np.round(np.sqrt(n)))
#     example_height = n / example_width

#     # Calcula el numero de elementos a mostrar
#     display_rows = int(np.floor(np.sqrt(m)))
#     display_cols = int(np.ceil(m / display_rows))

#     fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
#     fig.subplots_adjust(wspace=0.025, hspace=0.025)

#     ax_array = [ax_array] if m == 1 else ax_array.ravel()

#     for i, ax in enumerate(ax_array):
#         ax.imshow(X[i].reshape(example_width, example_width, order='F'),
#                   cmap='Greys', extent=[0, 1, 0, 1])
#         ax.axis('off')

In [23]:
# # Selecciona aleatoriamente 100 puntos de datos para mostrar
# rand_indices = np.random.choice(m, 100, replace=False)
# sel = X.iloc[rand_indices, :]

# displayData(sel)

In [24]:
#definimos la funcion de la sigmoide
import numpy as np
def sigmoid(z):
    # clip the input values to a smaller range to avoid overflow
    z = np.clip(z, -500, 500)
    return 1.0 / (1.0 + np.exp(-z))

In [25]:
print(sigmoid(0))

0.5


In [26]:
def lrCostFunction(theta, X, y, lambda_):
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))

    # J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    J = (1 / m) * np.sum(-y.dot(np.log(h + 1e-10)) - (1 - y).dot(np.log(1 - h + 1e-10))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

####  Vectorización regularizada de la regresión logística


<a id="section2"></a>
### Clasificacion One-vs-all


In [27]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [28]:
lambda_ = 0.6
all_theta = oneVsAll(X_norm, y_train, num_labels, lambda_)
# print(all_theta.shape)


In [29]:
for i in range(all_theta.shape[1]):
    print("Theta para la clase 0", i, " es: ", all_theta[0][i], "Theta para la clase 1", i, " es: ", all_theta[1][i], "Theta para la clase 2", i, " es: ", all_theta[2][i])

Theta para la clase 0 0  es:  1.016499690011147 Theta para la clase 1 0  es:  -4.294004940321746 Theta para la clase 2 0  es:  -6.256612484101175
Theta para la clase 0 1  es:  0.338612701372758 Theta para la clase 1 1  es:  0.7870726293312912 Theta para la clase 2 1  es:  -0.4260722898559919
Theta para la clase 0 2  es:  0.004501426207969591 Theta para la clase 1 2  es:  -0.02603871726870147 Theta para la clase 2 2  es:  0.017300217077188037
Theta para la clase 0 3  es:  0.1110594017731684 Theta para la clase 1 3  es:  -0.003215924858656367 Theta para la clase 2 3  es:  -0.10140655068351721
Theta para la clase 0 4  es:  0.2837441347285627 Theta para la clase 1 4  es:  -0.8206791633859548 Theta para la clase 2 4  es:  -0.5705007022957806
Theta para la clase 0 5  es:  -1.4342679091167971 Theta para la clase 1 5  es:  3.881600483253546 Theta para la clase 2 5  es:  0.4311844076267437
Theta para la clase 0 6  es:  -1.2980584625943234 Theta para la clase 1 6  es:  -0.31139233566084273 Theta

<a id="section3"></a>
#### Prediccion One-vs-all


In [30]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

Calculando la presicion

In [31]:
#Realiamos la prediccion y evaluamos la precision para un modelo de regresion logistica
print(X.shape)

pred = predictOneVsAll(all_theta, X_norm)
print('Precision del conjuto de entrenamiento haciendo uso del 80% de datos: {:.2f}%'.format(np.mean(pred == y_train) * 100))

X_test_norm = (X_test - mu) / sigma
p = predictOneVsAll(all_theta, X_test_norm)
print('Precision del conjuto de entrenamiento haciendo uso de 20% de datos: {:.2f}%'.format(np.mean(p == y_test) * 100))


for i in range(30):
    print("Prediccion: ", p[i], "Valor real: ", y_test[i])
    
# XPrueba = X.iloc[10:150, :].copy()
# print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
# XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
# print(XPrueba.shape)
# p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
# print(p)

# # displayData(X[1002:1003, :])
# print(y[10:150])

(79999, 40)
Precision del conjuto de entrenamiento haciendo uso del 80% de datos: 98.83%
Precision del conjuto de entrenamiento haciendo uso de 20% de datos: 98.82%
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  2 Valor real:  2
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  2 Valor real:  2
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  2 Valor real:  2
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  1 Valor real:  1
Prediccion:  2 Valor real:  2
Prediccion:  2 Valor real:  2
Prediccion:  1 Valor real